In [1]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from torchvision import transforms

from trainer import fit
import numpy as np

cuda = torch.cuda.is_available()

# TRIPLET

In [2]:
# Set up data loaders
from datasets import TripletDataset

root_dir = '/home/cuong/AIC20-Track2/AIC20_track2/AIC20_ReID/image_train'
train_csv = 'cls_train.csv'
val_csv = 'cls_val.csv'
label_json = 'train_image_metadata.json'

size = (224, 224)

triplet_train_dataset = TripletDataset(root_dir, train_csv, label_json,
                                       transform = transforms.Compose([
                                        transforms.Resize(size),  
                                        transforms.ToTensor()
                                      ]))
triplet_val_dataset = TripletDataset(root_dir, val_csv, label_json,
                                     transform = transforms.Compose([
                                        transforms.Resize(size),
                                        transforms.ToTensor()
                                      ]))

batch_size = 8
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
triplet_val_loader = torch.utils.data.DataLoader(triplet_val_dataset, batch_size=batch_size, shuffle=False, **kwargs)

In [3]:
# Set up the network and training parameters
from networks import EfficientNetExtractor, TripletNet
from losses import TripletLoss

margin = 1.
embedding_net = EfficientNetExtractor('b4')
model = TripletNet(embedding_net)

if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

Loaded pretrained weights for efficientnet-b4


In [ ]:
fit(triplet_train_loader, triplet_val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

/home/cuong/AIC20-Track2/venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/25585 (0%)]	Loss: 24.244934


In [ ]:
torch.save(model, 'triplet-b4-200404.pth')

# ONLINE TRIPLET LOSS

In [2]:
# Set up data loaders
from datasets import NormalDataset
from datasets import BalancedBatchSampler

root_dir = '/home/cuong/AIC20-Track2/AIC20_track2/AIC20_ReID/image_train'
train_csv = 'cls_train.csv'
val_csv = 'cls_val.csv'
label_json = 'train_image_metadata.json'

size = (224, 224)

train_dataset = NormalDataset(root_dir, train_csv, label_json,
                                       transform = transforms.Compose([
                                        transforms.Resize(size),  
                                        transforms.ToTensor()
                                      ]))
val_dataset = NormalDataset(root_dir, val_csv, label_json,
                                     transform = transforms.Compose([
                                        transforms.Resize(size),
                                        transforms.ToTensor()
                                      ]))



train_batch_sampler = BalancedBatchSampler(train_dataset.labels, n_classes=5, n_samples=5)
val_batch_sampler = BalancedBatchSampler(val_dataset.labels, n_classes=5, n_samples=5)

kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
online_train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=train_batch_sampler, **kwargs)
online_val_loader = torch.utils.data.DataLoader(val_dataset, batch_sampler=val_batch_sampler, **kwargs)

In [3]:
# Set up the network and training parameters
from networks import EfficientNetExtractor
from losses import OnlineTripletLoss
from utils import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from metrics import AverageNonzeroTripletsMetric

margin = 1.
embedding_net = EfficientNetExtractor('b4')
model = embedding_net

if cuda:
    model.cuda()
loss_fn = OnlineTripletLoss(margin, RandomNegativeTripletSelector(margin))
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 20
log_interval = 100

Loaded pretrained weights for efficientnet-b4


In [ ]:
fit(online_train_loader, online_val_loader, model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval, metrics=[AverageNonzeroTripletsMetric()])

/home/cuong/AIC20-Track2/venv/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train: [0/25585 (0%)]	Loss: 26.806299	Average nonzero triplets: 45.0
Train: [2500/25585 (10%)]	Loss: 7.513605	Average nonzero triplets: 39.21782178217822
Train: [5000/25585 (20%)]	Loss: 3.312215	Average nonzero triplets: 34.37313432835821
Train: [7500/25585 (29%)]	Loss: 3.564842	Average nonzero triplets: 30.78737541528239
Train: [10000/25585 (39%)]	Loss: 3.150719	Average nonzero triplets: 28.2643391521197
Train: [12500/25585 (49%)]	Loss: 3.583415	Average nonzero triplets: 26.780439121756487
Train: [15000/25585 (59%)]	Loss: 2.783527	Average nonzero triplets: 25.420965058236273
Train: [17500/25585 (68%)]	Loss: 2.617585	Average nonzero triplets: 24.332382310984308
Train: [20000/25585 (78%)]	Loss: 2.347412	Average nonzero triplets: 23.409488139825218
Train: [22500/25585 (88%)]	Loss: 2.287711	Average nonzero triplets: 22.69589345172031
Train: [25000/25585 (98%)]	Loss: 2.089530	Average nonzero triplets: 21.972027972027973
Epoch: 1/20. Train set: Average loss: 3.3215	Average nonzero triplets:

In [ ]:
torch.save(model, 'onlinetriplet-b4-200404.pth')